# This is to show and test my abilities to actually create a notebook and test my skills I learned from my Data Science & ML Courses 
This notebook is for everyone to see and comment on and is a test of my skills along with some research, feel free to tell me my mistakes and what I should've done, and could do better ❤

## 1. Problem Definition

> Predict target values and submit the score on Kaggle


# 2. Data
As we could see from the data this is a classification problem, some of the columns have string values we must convert into categorical values.

> For this competition, you will be predicting a binary target based on a number of feature columns given in the data. All of the feature columns, cat0 - cat18 are categorical, and the feature columns cont0 - cont10 are continuous.  
Files  
train.csv - the training data with the target column  
test.csv - the test set; you will be predicting the target for each row in this file (the probability of the binary target)  
sample_submission.csv - a sample submission file in the correct format



In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score

import pickle

In [36]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [37]:
train_df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


## Now we're going to convert the categorical values and assume they are ordinal

When to use a Label Encoding vs. One Hot Encoding
This question generally depends on your dataset and the model which you wish to apply. But still, a few points to note before choosing the right encoding technique for your model:


> **We apply One-Hot Encoding when:**  
◽ The categorical feature is not ordinal (like the countries above)  
◽ The number of categorical features is less so one-hot encoding can be effectively applied  
**We apply Label Encoding when:**  
◽ The categorical feature is ordinal (like Jr. kg, Sr. kg, Primary school, high school)  
◽ The number of categories is quite large as one-hot encoding can lead to high memory consumption  
link: https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/  

We are also dropping the ID column above, this should help in getting optimal results rather than having our algorithims see them as values and have them result as outliers

In [38]:
# create a list and store the columns
cat_cols = []
for col in train_df.columns:
    cat_cols.append(col)

# use only the categorical columns    
cat_cols = cat_cols[1:20]

# setup LabelEncoder
label_encoder = LabelEncoder()

# loop through the dataset's columns and transform their values into numerical values
for col in cat_cols:
    test_df[col] = label_encoder.fit_transform(test_df[col])
    train_df[col] = label_encoder.fit_transform(train_df[col])
    
# set random seed to reproduce
np.random.seed(42)

# shuffle the dataset and drop the id column
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df = train_df.drop(columns='id', axis=1)
test_df = test_df.drop(columns='id', axis=1)

# split the data
X = train_df.drop('target', axis=1)
y = train_df.target

# split the data into train, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Since we're dealing with a classification problem we're going to try three different models to try and compare before we get to the experimentation part



In [5]:
# we're going to create a dict of the different models we're going to use
models = {
    'Logistic Regression' : LogisticRegression(solver='liblinear'),
    'RandomForest' : RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
}

In [6]:
# create a function where we loop through the dict and use score them
def fit_and_score(models, X_train, X_test, y_train, y_test):
    model_scores = {}
    for name, model in models.items(): 
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [7]:
model_scores = fit_and_score(models=models, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
model_scores

{'Logistic Regression': 0.8376222222222223,
 'RandomForest': 0.8464,
 'KNN': 0.8109222222222222}

Based on the score above we're going to use RandomForestClassifier, unless we have a lot of time to actually test out and experiment on these algorithms having the highest one would be time efficient(unless we have a good cpu). 

# 3. Experimentation

## We're then going to manually test out hyperparameters
this is the part where we're going to experiment a lot on, set yourself a limit on the result you want or you'll never stop training, with this amount of dataset and amount of combinations and fine-tuning your hyperparameter may take you a lot of time.

for the meaning of these hyperparameters, go to sklearn's documentation page for more information

In [8]:
rf_grid = {
    'n_estimators' : np.arange(600, 1500, 50),
    'max_depth' : [None, 1,3,5],
    'max_features': ['auto'],
    'min_samples_split' : np.arange(1,15,1),
    'min_samples_leaf' : np.arange(1,15,1),
    'bootstrap' : [True ,False]
}

### it is way better to train on your own PC/locally since you can maximize the amount of threads you have especially if you have a mid-end PC 

In [9]:
rs_rf = RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions=rf_grid,
                          cv=5,
                          n_iter=5,
                          verbose=2,
                          n_jobs=-1
                          )
rs_rf.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  25 | elapsed: 21.9min remaining: 14.6min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 24.9min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 1, 3, 5],
                                        'max_features': ['auto'],
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_samples_split': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'n_estimators': array([ 600,  650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100,
       1150, 1200, 1250, 1300, 1350, 1400, 1450])},
                   verbose=2)

#### now to check on the best parameters our RandomizedSearchCV has found

In [10]:
rs_rf.best_params_

{'n_estimators': 1200,
 'min_samples_split': 6,
 'min_samples_leaf': 10,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

In [11]:
rs_y_preds_proba = rs_rf.predict_proba(X_test)[:,1]
score = roc_auc_score(y_test, rs_y_preds_proba)
print(f'{score:0.5f}')

0.88822


#### We then store our result to a file using pickle just incase something happens(I do this since my electricity goes out a lot)

In [12]:
# load pickle
load_pickle_model = pickle.load(open('rs_random_forest_model.pkl', 'rb'))

### We then use our saved model to predict on the test dataset we were given, and then save it, and then we submit our results to the kaggle page

In [13]:
pickle_y_preds = load_pickle_model.predict_proba(test_df)[:,1]
submission_df['target'] = pickle_y_preds
submission_df.to_csv('random_forest_rs.csv', index=False)

## Now if we wanna go a bit further and fine-tune our model, we can use GridSearchCV
GridSearchCV actually takes time since it doesn't randomly choose a model and skip it  
but instead it actually tries every combination you have on your parameters dictionary

In [14]:
gs_grid = {
    'n_estimators' : np.arange(1100, 1350, 50),
    'max_depth' : [None],
    'max_features': ['auto'],
    'min_samples_split' : np.arange(8,10,1),
    'min_samples_leaf' : np.arange(12,14,1),
    'bootstrap' : [False]
}

##### Do Note: I got these values are from the RandomizedSearchCV best params and created a range of values to find a somewhat sweetspot.

In [15]:
gs_rf = GridSearchCV(RandomForestClassifier(),
                          param_grid=gs_grid,
                          cv=5,
                          verbose=2,
                          n_jobs=-1
                          )
gs_rf.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 38.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 169.5min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [None],
                         'max_features': ['auto'],
                         'min_samples_leaf': array([12, 13]),
                         'min_samples_split': array([8, 9]),
                         'n_estimators': array([1100, 1150, 1200, 1250, 1300])},
             verbose=2)

In [6]:
# save pickle
pickle.dump(gs_rf, open('gs_random_forest_model.pkl', 'wb'))


In [39]:
# load pickle
load_pickle_model_gs = pickle.load(open('gs_random_forest_model.pkl', 'rb'))

In [40]:
load_pickle_model_gs.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 12,
 'min_samples_split': 8,
 'n_estimators': 1200}

In [41]:
pickle_y_preds_gs = load_pickle_model_gs.predict(test_df)
submission_df['target'] = pickle_y_preds_gs
submission_df.to_csv('random_forest.csv', index=False)

### After submission on Kaggle's competition page, we're going to check the results if our model predicts well... we're then going to use those as pseudolabels for out next training which is a form of blending. 

In [97]:
pseudo_df = pd.read_csv('random_forest_psuedo.csv')

In [103]:
 pseudo_df['target'].round()

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
199995    1.0
199996    0.0
199997    1.0
199998    0.0
199999    0.0
Name: target, Length: 200000, dtype: float64

In [104]:
test_df['target'] = pseudo_df['target'].round()
all_df = pd.concat([train_df, test_df],axis=0)

In [105]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score
import pickle

In [107]:
# create a list and store the columns
cat_cols = []
for col in all_df.columns:
    cat_cols.append(col)

# use only the categorical columns    
cat_cols = cat_cols[1:20]

# setup LabelEncoder
label_encoder = LabelEncoder()

# loop through the dataset's columns and transform their values into numerical values
for col in cat_cols:
    test_df[col] = label_encoder.fit_transform(test_df[col])
    all_df[col] = label_encoder.fit_transform(all_df[col])
    
# set random seed to reproduce
np.random.seed(42)

# shuffle the dataset and drop the id column
all_df = all_df.sample(frac=1).reset_index(drop=True)


In [108]:
all_df

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,11,2,0,4,33,0,30,60,0,...,0.335832,0.253207,0.212027,0.418792,0.352532,0.338620,0.371403,0.374276,0.332941,0.0
1,0,11,0,1,7,33,0,49,3,0,...,0.805537,0.700713,0.666567,0.680748,0.302710,0.726929,0.578570,0.820023,0.545562,0.0
2,1,6,0,1,4,2,2,20,43,0,...,0.447609,0.338504,0.424206,0.735625,0.149244,0.587069,0.357628,0.485870,0.589181,0.0
3,0,11,8,1,4,33,0,47,60,0,...,0.598023,0.774466,0.606669,0.070092,0.163243,0.838868,0.604804,0.526557,0.925181,1.0
4,1,8,3,0,4,33,2,16,38,0,...,0.390959,0.317699,0.338363,0.555064,0.556805,0.385767,0.360034,0.320340,0.513304,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,13,0,2,4,33,7,32,5,0,...,0.940246,0.279625,0.211738,0.682384,0.594475,0.399542,0.476373,0.285167,0.391800,0.0
499996,0,11,8,1,4,33,0,8,60,0,...,0.848035,0.827115,0.208683,0.552015,0.257078,0.910031,0.615789,0.841467,0.947278,0.0
499997,0,10,0,1,4,33,0,30,38,5,...,0.302326,0.559245,0.816425,0.486430,0.444094,0.244764,0.340735,0.249448,0.774085,0.0
499998,0,13,11,1,4,45,2,19,55,0,...,0.254586,0.680934,0.604287,0.291691,0.175051,0.357269,0.293581,0.255408,0.683376,0.0


In [109]:
# split the target column and features column
X = all_df.drop('target', axis=1)
y = all_df.target

# split using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [110]:
# we're going to create a dict of the different models we're going to use
models = {
    'Logistic Regression' : LogisticRegression(solver='liblinear'),
    'Ridge Classifier' : RidgeClassifier(),
    'SGDClassifier' : SGDClassifier(),
    'RandomForest' : RandomForestClassifier(),
    'ExtraTreesClassifier' : ExtraTreesClassifier(),
    'KNC': KNeighborsClassifier(),
    'DecisionTreeClassifier':DecisionTreeClassifier(),
    'LGBMClassifier' : LGBMClassifier(verbose=-1),
    'XGBClassifier' : XGBClassifier(verbose=0, use_label_encoder=False,),
    'CatBoostClassifier' : CatBoostClassifier(verbose=0),
}

In [111]:
# create a function where we loop through the dict and use score them
def fit_and_score(models, X_train, X_test, y_train, y_test):
    model_scores = {}
    for name, model in models.items(): 
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [112]:
model_scores = fit_and_score(models=models, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
model_scores

[15:04:11] WARNING: ..\src\learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:04:12] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'Logistic Regression': 0.76112,
 'Ridge Classifier': 0.8818466666666667,
 'SGDClassifier': 0.7576666666666667,
 'RandomForest': 0.8993933333333334,
 'ExtraTreesClassifier': 0.8985066666666667,
 'KNC': 0.73484,
 'DecisionTreeClassifier': 0.8455133333333333,
 'LGBMClassifier': 0.8997933333333333,
 'XGBClassifier': 0.9011866666666667,
 'CatBoostClassifier': 0.9037666666666667}

In [113]:
import operator
max(model_scores.items(), key=operator.itemgetter(1))[0]

'CatBoostClassifier'

In [55]:
test_df = test_df.drop('target', axis =1)

In [115]:
dtm_oof = np.zeros(train_df.shape[0])
dtm_preds = np.zeros(test_df.shape[0])
feature_importances = pd.DataFrame()


skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(all_df, all_df['target'])):
    print(f"===== FOLD {fold} =====")
    oof_idx = np.array([idx for idx in valid_idx if idx < train_df.shape[0]])
    preds_idx = np.array([idx for idx in valid_idx if idx >= train_df.shape[0]])

    X_train, y_train = all_df.iloc[train_idx].drop('target', axis=1), all_df.iloc[train_idx]['target']
    X_valid, y_valid = all_df.iloc[oof_idx].drop('target', axis=1), all_df.iloc[oof_idx]['target']
    X_test = all_df.iloc[preds_idx].drop('target', axis=1)
    
    model = CatBoostClassifier(
    )
    model.fit(X_train, y_train)
    
    dtm_oof[oof_idx] = model.predict_proba(X_valid)[:,1]
    dtm_preds[preds_idx-train_df.shape[0]] = model.predict_proba(X_test)[:,1]
    
    acc_score = accuracy_score(y_valid, np.where(dtm_oof[oof_idx]>0.5, 1, 0))
    print(f"===== ACCURACY SCORE {acc_score:.6f} =====\n")
    
acc_score = accuracy_score(all_df[:train_df.shape[0]]['target'], np.where(dtm_oof>0.5, 1, 0))
print(f"===== ACCURACY SCORE {acc_score:.6f} =====")

===== FOLD 0 =====
Learning rate set to 0.139912
0:	learn: 0.5507137	total: 43.1ms	remaining: 43s
1:	learn: 0.4714807	total: 82.6ms	remaining: 41.2s
2:	learn: 0.4195330	total: 124ms	remaining: 41.3s
3:	learn: 0.3727952	total: 165ms	remaining: 41.1s
4:	learn: 0.3490234	total: 208ms	remaining: 41.4s
5:	learn: 0.3327663	total: 248ms	remaining: 41.1s
6:	learn: 0.3208566	total: 287ms	remaining: 40.7s
7:	learn: 0.3131522	total: 327ms	remaining: 40.6s
8:	learn: 0.3022652	total: 372ms	remaining: 40.9s
9:	learn: 0.2972985	total: 416ms	remaining: 41.1s
10:	learn: 0.2935143	total: 464ms	remaining: 41.7s
11:	learn: 0.2878291	total: 512ms	remaining: 42.1s
12:	learn: 0.2852528	total: 558ms	remaining: 42.4s
13:	learn: 0.2831767	total: 605ms	remaining: 42.6s
14:	learn: 0.2786504	total: 650ms	remaining: 42.7s
15:	learn: 0.2771507	total: 692ms	remaining: 42.5s
16:	learn: 0.2743773	total: 737ms	remaining: 42.6s
17:	learn: 0.2730243	total: 786ms	remaining: 42.9s
18:	learn: 0.2720222	total: 825ms	remaining

160:	learn: 0.2359185	total: 7.22s	remaining: 37.6s
161:	learn: 0.2358529	total: 7.26s	remaining: 37.6s
162:	learn: 0.2357795	total: 7.31s	remaining: 37.5s
163:	learn: 0.2356606	total: 7.35s	remaining: 37.5s
164:	learn: 0.2355829	total: 7.39s	remaining: 37.4s
165:	learn: 0.2355106	total: 7.43s	remaining: 37.3s
166:	learn: 0.2354043	total: 7.48s	remaining: 37.3s
167:	learn: 0.2353291	total: 7.52s	remaining: 37.3s
168:	learn: 0.2352506	total: 7.57s	remaining: 37.2s
169:	learn: 0.2351512	total: 7.62s	remaining: 37.2s
170:	learn: 0.2350176	total: 7.66s	remaining: 37.1s
171:	learn: 0.2349399	total: 7.7s	remaining: 37.1s
172:	learn: 0.2348567	total: 7.74s	remaining: 37s
173:	learn: 0.2347836	total: 7.78s	remaining: 36.9s
174:	learn: 0.2347047	total: 7.83s	remaining: 36.9s
175:	learn: 0.2346010	total: 7.87s	remaining: 36.8s
176:	learn: 0.2345425	total: 7.91s	remaining: 36.8s
177:	learn: 0.2344922	total: 7.95s	remaining: 36.7s
178:	learn: 0.2344193	total: 7.99s	remaining: 36.6s
179:	learn: 0.2

322:	learn: 0.2255912	total: 14.6s	remaining: 30.6s
323:	learn: 0.2255538	total: 14.6s	remaining: 30.5s
324:	learn: 0.2255173	total: 14.7s	remaining: 30.5s
325:	learn: 0.2254799	total: 14.7s	remaining: 30.5s
326:	learn: 0.2254436	total: 14.8s	remaining: 30.4s
327:	learn: 0.2254017	total: 14.8s	remaining: 30.4s
328:	learn: 0.2253483	total: 14.9s	remaining: 30.3s
329:	learn: 0.2253101	total: 14.9s	remaining: 30.3s
330:	learn: 0.2252732	total: 15s	remaining: 30.2s
331:	learn: 0.2252398	total: 15s	remaining: 30.2s
332:	learn: 0.2252031	total: 15s	remaining: 30.1s
333:	learn: 0.2251397	total: 15.1s	remaining: 30.1s
334:	learn: 0.2250871	total: 15.1s	remaining: 30s
335:	learn: 0.2250261	total: 15.2s	remaining: 30s
336:	learn: 0.2249858	total: 15.2s	remaining: 29.9s
337:	learn: 0.2249509	total: 15.3s	remaining: 29.9s
338:	learn: 0.2249138	total: 15.3s	remaining: 29.8s
339:	learn: 0.2248626	total: 15.3s	remaining: 29.8s
340:	learn: 0.2248165	total: 15.4s	remaining: 29.7s
341:	learn: 0.2247405	

484:	learn: 0.2191541	total: 21.7s	remaining: 23s
485:	learn: 0.2190884	total: 21.7s	remaining: 23s
486:	learn: 0.2190600	total: 21.8s	remaining: 22.9s
487:	learn: 0.2190335	total: 21.8s	remaining: 22.9s
488:	learn: 0.2190093	total: 21.8s	remaining: 22.8s
489:	learn: 0.2189693	total: 21.9s	remaining: 22.8s
490:	learn: 0.2189516	total: 21.9s	remaining: 22.7s
491:	learn: 0.2189172	total: 22s	remaining: 22.7s
492:	learn: 0.2188940	total: 22s	remaining: 22.6s
493:	learn: 0.2188556	total: 22.1s	remaining: 22.6s
494:	learn: 0.2188185	total: 22.1s	remaining: 22.6s
495:	learn: 0.2187882	total: 22.2s	remaining: 22.5s
496:	learn: 0.2187628	total: 22.2s	remaining: 22.5s
497:	learn: 0.2187395	total: 22.2s	remaining: 22.4s
498:	learn: 0.2187062	total: 22.3s	remaining: 22.4s
499:	learn: 0.2186825	total: 22.3s	remaining: 22.3s
500:	learn: 0.2186682	total: 22.4s	remaining: 22.3s
501:	learn: 0.2186314	total: 22.4s	remaining: 22.2s
502:	learn: 0.2185786	total: 22.5s	remaining: 22.2s
503:	learn: 0.218543

645:	learn: 0.2139951	total: 28.7s	remaining: 15.7s
646:	learn: 0.2139752	total: 28.8s	remaining: 15.7s
647:	learn: 0.2139504	total: 28.8s	remaining: 15.6s
648:	learn: 0.2139240	total: 28.9s	remaining: 15.6s
649:	learn: 0.2138944	total: 28.9s	remaining: 15.6s
650:	learn: 0.2138707	total: 28.9s	remaining: 15.5s
651:	learn: 0.2138402	total: 29s	remaining: 15.5s
652:	learn: 0.2138179	total: 29s	remaining: 15.4s
653:	learn: 0.2137924	total: 29.1s	remaining: 15.4s
654:	learn: 0.2137726	total: 29.1s	remaining: 15.3s
655:	learn: 0.2137312	total: 29.1s	remaining: 15.3s
656:	learn: 0.2137011	total: 29.2s	remaining: 15.2s
657:	learn: 0.2136706	total: 29.2s	remaining: 15.2s
658:	learn: 0.2136465	total: 29.3s	remaining: 15.2s
659:	learn: 0.2136149	total: 29.3s	remaining: 15.1s
660:	learn: 0.2135689	total: 29.4s	remaining: 15.1s
661:	learn: 0.2135390	total: 29.4s	remaining: 15s
662:	learn: 0.2135117	total: 29.5s	remaining: 15s
663:	learn: 0.2134541	total: 29.5s	remaining: 14.9s
664:	learn: 0.213428

...

340:	learn: 0.2246679	total: 14.9s	remaining: 28.7s
341:	learn: 0.2246237	total: 14.9s	remaining: 28.7s
342:	learn: 0.2245614	total: 14.9s	remaining: 28.6s
343:	learn: 0.2245024	total: 15s	remaining: 28.6s
344:	learn: 0.2244582	total: 15s	remaining: 28.5s
345:	learn: 0.2244199	total: 15.1s	remaining: 28.5s
346:	learn: 0.2243866	total: 15.1s	remaining: 28.4s
347:	learn: 0.2243366	total: 15.2s	remaining: 28.4s
348:	learn: 0.2243152	total: 15.2s	remaining: 28.3s
349:	learn: 0.2242755	total: 15.2s	remaining: 28.3s
350:	learn: 0.2242432	total: 15.3s	remaining: 28.3s
351:	learn: 0.2242006	total: 15.3s	remaining: 28.2s
352:	learn: 0.2241534	total: 15.4s	remaining: 28.2s
353:	learn: 0.2241123	total: 15.4s	remaining: 28.1s
354:	learn: 0.2240855	total: 15.4s	remaining: 28.1s
355:	learn: 0.2240338	total: 15.5s	remaining: 28s
356:	learn: 0.2239446	total: 15.5s	remaining: 28s
357:	learn: 0.2239147	total: 15.6s	remaining: 27.9s
358:	learn: 0.2238534	total: 15.6s	remaining: 27.9s
359:	learn: 0.223814

502:	learn: 0.2183303	total: 22s	remaining: 21.8s
503:	learn: 0.2182991	total: 22.1s	remaining: 21.7s
504:	learn: 0.2182485	total: 22.1s	remaining: 21.7s
505:	learn: 0.2182016	total: 22.2s	remaining: 21.6s
506:	learn: 0.2181675	total: 22.2s	remaining: 21.6s
507:	learn: 0.2181459	total: 22.2s	remaining: 21.5s
508:	learn: 0.2181186	total: 22.3s	remaining: 21.5s
509:	learn: 0.2180915	total: 22.3s	remaining: 21.5s
510:	learn: 0.2180690	total: 22.4s	remaining: 21.4s
511:	learn: 0.2180446	total: 22.4s	remaining: 21.4s
512:	learn: 0.2180130	total: 22.5s	remaining: 21.3s
513:	learn: 0.2179788	total: 22.5s	remaining: 21.3s
514:	learn: 0.2179473	total: 22.5s	remaining: 21.2s
515:	learn: 0.2179230	total: 22.6s	remaining: 21.2s
516:	learn: 0.2178958	total: 22.6s	remaining: 21.2s
517:	learn: 0.2178440	total: 22.7s	remaining: 21.1s
518:	learn: 0.2178095	total: 22.7s	remaining: 21.1s
519:	learn: 0.2177518	total: 22.8s	remaining: 21s
520:	learn: 0.2177253	total: 22.8s	remaining: 21s
521:	learn: 0.2176

662:	learn: 0.2134843	total: 29.2s	remaining: 14.8s
663:	learn: 0.2134589	total: 29.2s	remaining: 14.8s
664:	learn: 0.2134288	total: 29.3s	remaining: 14.7s
665:	learn: 0.2133936	total: 29.3s	remaining: 14.7s
666:	learn: 0.2133607	total: 29.3s	remaining: 14.6s
667:	learn: 0.2133140	total: 29.4s	remaining: 14.6s
668:	learn: 0.2132883	total: 29.4s	remaining: 14.6s
669:	learn: 0.2132570	total: 29.5s	remaining: 14.5s
670:	learn: 0.2132265	total: 29.5s	remaining: 14.5s
671:	learn: 0.2132056	total: 29.6s	remaining: 14.4s
672:	learn: 0.2131634	total: 29.6s	remaining: 14.4s
673:	learn: 0.2131379	total: 29.6s	remaining: 14.3s
674:	learn: 0.2131118	total: 29.7s	remaining: 14.3s
675:	learn: 0.2130874	total: 29.7s	remaining: 14.3s
676:	learn: 0.2130540	total: 29.8s	remaining: 14.2s
677:	learn: 0.2130312	total: 29.8s	remaining: 14.2s
678:	learn: 0.2130094	total: 29.9s	remaining: 14.1s
679:	learn: 0.2129818	total: 29.9s	remaining: 14.1s
680:	learn: 0.2129579	total: 30s	remaining: 14s
681:	learn: 0.21

821:	learn: 0.2093778	total: 36.4s	remaining: 7.88s
822:	learn: 0.2093564	total: 36.4s	remaining: 7.83s
823:	learn: 0.2093347	total: 36.5s	remaining: 7.79s
824:	learn: 0.2093083	total: 36.5s	remaining: 7.75s
825:	learn: 0.2092854	total: 36.6s	remaining: 7.7s
826:	learn: 0.2092626	total: 36.6s	remaining: 7.66s
827:	learn: 0.2092473	total: 36.6s	remaining: 7.61s
828:	learn: 0.2092175	total: 36.7s	remaining: 7.57s
829:	learn: 0.2091931	total: 36.7s	remaining: 7.52s
830:	learn: 0.2091719	total: 36.8s	remaining: 7.48s
831:	learn: 0.2091530	total: 36.9s	remaining: 7.44s
832:	learn: 0.2091288	total: 36.9s	remaining: 7.41s
833:	learn: 0.2091037	total: 37s	remaining: 7.37s
834:	learn: 0.2090781	total: 37.1s	remaining: 7.33s
835:	learn: 0.2090517	total: 37.2s	remaining: 7.29s
836:	learn: 0.2090233	total: 37.2s	remaining: 7.25s
837:	learn: 0.2089927	total: 37.3s	remaining: 7.21s
838:	learn: 0.2089680	total: 37.4s	remaining: 7.17s
839:	learn: 0.2089447	total: 37.5s	remaining: 7.13s
840:	learn: 0.2

984:	learn: 0.2055179	total: 44s	remaining: 669ms
985:	learn: 0.2055082	total: 44s	remaining: 625ms
986:	learn: 0.2054885	total: 44s	remaining: 580ms
987:	learn: 0.2054557	total: 44.1s	remaining: 536ms
988:	learn: 0.2054356	total: 44.1s	remaining: 491ms
989:	learn: 0.2054135	total: 44.2s	remaining: 446ms
990:	learn: 0.2053944	total: 44.2s	remaining: 402ms
991:	learn: 0.2053652	total: 44.3s	remaining: 357ms
992:	learn: 0.2053424	total: 44.3s	remaining: 312ms
993:	learn: 0.2053149	total: 44.4s	remaining: 268ms
994:	learn: 0.2052981	total: 44.4s	remaining: 223ms
995:	learn: 0.2052718	total: 44.4s	remaining: 178ms
996:	learn: 0.2052520	total: 44.5s	remaining: 134ms
997:	learn: 0.2052299	total: 44.5s	remaining: 89.2ms
998:	learn: 0.2052068	total: 44.6s	remaining: 44.6ms
999:	learn: 0.2051846	total: 44.6s	remaining: 0us
===== ACCURACY SCORE 0.904775 =====

===== ACCURACY SCORE 0.904847 =====


In [117]:
submission_df['target'] = model.predict_proba(test_df)[:,1]
submission_df.to_csv('cat_boost_blending.csv', index=False)

Special Thanks to 

https://www.kaggle.com/inversion/get-started-mar-tabular-playground-competition  
https://www.kaggle.com/hiro5299834/tps-apr-2021-pseudo-labeling-voting-ensemble#LightGBM

Stuff to do next:

RandomizedSearchCV